In [ ]:
import os
import requests
import pandas as pd
from tqdm import tqdm
from io import StringIO
from tqdm import tqdm_notebook as tqdm_nb

In [ ]:
molecule = []
iso_slug = []
isotopologue = []
path_mol_iso = []
for line in open('./data/url/api__urls.txt'):
    molecule.append(line.split('/')[-4])
    iso_slug.append(line.split('/')[-3])
    isotopologue.append(line.split('/')[-2])
    path_mol_iso.append(line.split('/')[-4] + '/' + line.split('/')[-3] + '/' + line.split('/')[-2])

molecule_list = list(set(molecule))
molecule_list.sort(key=molecule.index)

iso_slug_list = list(set(iso_slug))
iso_slug_list.sort(key=iso_slug.index)

isotopologue_list = list(set(isotopologue))
isotopologue_list.sort(key=isotopologue.index)

path_mol_iso_list = list(set(path_mol_iso))
path_mol_iso_list.sort(key=path_mol_iso.index)

print('Molecule:', molecule_list)
print('Iso-slug:', iso_slug_list)
print('Isotopologue:', isotopologue_list)
print('Total:', path_mol_iso_list)


In [ ]:
states_trans_pf_path = './data/states_trans_pf/'
if os.path.exists(states_trans_pf_path):
    pass
else:
    os.makedirs(states_trans_pf_path, exist_ok=True)


states_shell = []
trans_shell = []
pf_col_name = ['T', 'Q']

for path_mol_iso in tqdm(path_mol_iso_list):
    
    de_path = './data/decompress/' + path_mol_iso
    
    save_states_path = states_trans_pf_path + path_mol_iso.replace('/','__') + '_states.txt'
    save_trans_path = states_trans_pf_path + path_mol_iso.replace('/','__') + '_trans.txt'
    save_pf_path = states_trans_pf_path + path_mol_iso.replace('/','__') + '_pf.csv'
    
    states_shell.append('cat ' + de_path + '/*states.inp > ' + save_states_path  + ' \n')
    trans_shell.append('cat ' + de_path + '/*trans.inp > ' + save_trans_path  + ' \n')

    pf_url = ('http://www.exomol.com/db/' + path_mol_iso + '/'
              + path_mol_iso.split('/')[1] + '__'
              + path_mol_iso.split('/')[2] + '.pf')   
    response = requests.get(pf_url)
    content = response.text  
    pf_data = pd.read_csv(StringIO(content), sep='\s+', names=pf_col_name, header=None, engine='python')
    pf_data.to_csv(save_pf_path, header=False)


In [ ]:
with open('states_trans.sh', 'w') as file:
    file.write('free -m \n')
    file.write('/bin/sync \n')
    file.write('/bin/echo 1 > /proc/sys/vm/drop_caches \n')
    file.write('free -m \n')
    for command in states_shell:
        file.write(command)
        
    for command in trans_shell:
        file.write(command)
